### Exercicio 3 - lista 4

Determinacao da quantidade de bits em n a partir do RA

In [296]:
# determine o bit length baseado no RA
my_ra = 228956 
bit_len = (my_ra % 100) + 100
print(bit_len)

156


### Definindo as funcoes auxiliares para o RSA e de geracao de chaves

In [297]:
def bin_to_dec(bin_vec = []):
    max = len(bin_vec)
    decimal = 0
    for i in range(0, max):
        decimal += bin_vec[i] * (2**i)
    return decimal

In [298]:
def get_random_prime(min, max):
    num = 0
    while not is_prime(num):
        num = ZZ.random_element(min, max)
    return num

In [299]:
def totient(p, q):
    return (p-1) * (q-1)

In [300]:
def find_encription_key(totient_phi):
    e = ZZ.random_element(totient_phi)
    while gcd(totient_phi, e) != 1:
        e = ZZ.random_element(1, totient_phi)
    return e

In [301]:
def find_decription_key(e, totient_phi):
    return e.inverse_mod(totient_phi)

In [302]:
def find_key_pairs(totient_phi):
    e = d = -1
    while d < 0:
        e = find_encription_key(totient_phi)
        d = find_decription_key(e, totient_phi)
    return [e,d]

In [303]:
e, d = find_key_pairs(euler_phi)
print(e)
print(d)

8267781152110277372591943534226164848743308451
1158261753593424820906387915389282691016565115


In [360]:
def generate_rsa_key(prime_bit_length):
    
    p = get_random_prime(2^(prime_bit_length -1) +10, 2^prime_bit_length -1) # the +10 is to improve the probability of n be exactly 156 bits large
    q = get_random_prime(2^(prime_bit_length -1) +10, 2^prime_bit_length -1)
    
    while(q == p):
        q = get_random_prime(2**(prime_bit_length -1) +10, 2**78 -1)
    
    n = p*q
    phi = totient(p,q)
    e, d = find_key_pairs(phi)
    
    assert(mod(e*d, phi) == 1)
    assert(e < n)
    assert(d < n)
    
    return {
        "p"   : p,
        "q"   : q,
        "n"   : n,
        "phi" : phi,
        "e"   : e,
        "d"   : d
    }

In [305]:
def rsa_encrypt_decrypt(message, key, n):
    return power_mod(message, key, n)

### i) Demonstrando o sigilo da mensagem

In [306]:
ana  = generate_rsa_key(bit_len // 2)
beto = generate_rsa_key(bit_len // 2)

In [307]:
ana

{'d': 30979444125301915631482090155800735912849934187,
 'e': 52663478733789447378168807058560777600423914659,
 'n': 72939402863335886860162409435487842329433753831,
 'p': 268796859830643408786989,
 'phi': 72939402863335886860161869283512816919002101464,
 'q': 271355115194767022865379}

In [308]:
beto

{'d': 33262565787506711230984166887553415872841259933,
 'e': 41808828065721814671432428783749834914951982021,
 'n': 46329590668309653860104949918845661438150639363,
 'p': 177671234916983718403399,
 'phi': 46329590668309653860104511487388697497884189528,
 'q': 260760222046956548046437}

In [309]:
print(int(ana['n']).bit_length())
print(int(beto['n']).bit_length())

156
156


In [319]:
ana_p = 228956
ana_c = rsa_encrypt_decrypt(ana_p, ana['e'], ana['n'])
print(ana_c == ana_p)

ana_p2 = rsa_encrypt_decrypt(ana_c, ana['d'], ana['n'])

print("Mensagem: " + str(ana_p))
print("Texto encriptado: " + str(ana_c))
print("Mensagem apos decriptada: " + str(ana_p2))

False
Mensagem: 228956
Texto encriptado: 24894157193031953816368855848521229276562118237
Mensagem apos decriptada: 228956


### ii) Agora vamos demonstrar como Beto pode aferir a autenticidade de uma mensagem enviada por Ana

Beto envia para Ana uma mensagem de desafio encriptado com a chave publica de Ana, o passo abaixo considera que Ana ja recebeu o desafio e pretende voltar a mensagem para beto encriptado com sua propria chave privada.

In [311]:
desafio_ana = 2 # desafio que Beto enviou a Ana

Para aferir a Autenticidade, Ana deve encriptar a mensagem com sua chave privada, e com a chave publica de Beto

In [320]:
c_ana = rsa_encrypt_decrypt(desafio_ana, ana['d'], ana['n'])
c_ana

44828843197980668851445548372250029310416609519

In [321]:
c_ana_auth = rsa_encrypt_decrypt(c_ana, beto['e'], beto['n'])
c_ana_auth

3183998536260773381802859317839949492208698823

Para recurperar a mensagem e garantir sua autenticidade beto deve decripta-la com sua chave Privada e entao tentar decriptar com a chave pulica de Ana, se o processo for bem sucedido, entao significa que a mensagem eh autentica de Ana

In [322]:
p_ana_beto_auth = rsa_encrypt_decrypt(c_ana_auth, beto['d'], beto['n'])
p_ana_beto_auth

44828843197980668851445548372250029310416609519

In [323]:
p_ana_beto = rsa_encrypt_decrypt(p_ana_beto_auth, ana['e'], ana['n'])
p_ana_beto

2

Observa-se que Beto recuperou o conteudo da mensagem contendo o valor 2, identico ao desafio que foi enviado para Ana, ou seja, a autenticidade de Ana eh verdadeira

## Exercicio 8
#### Fatoracao utilizando PARI, QSieve e ECM
1) Diferenca de cada metodo, explicada no pdf principal.
2) Abaixo segue a fatoracao de n usando cada um dos metodos

In [353]:
n =  ana['n']
n

72939402863335886860162409435487842329433753831

In [408]:
time(n.factor())

CPU times: user 590 ms, sys: 16.5 ms, total: 606 ms
Wall time: 607 ms


268796859830643408786989 * 271355115194767022865379

In [409]:
time(n.factor(algorithm="qsieve"))

CPU times: user 3.02 ms, sys: 13.4 ms, total: 16.4 ms
Wall time: 761 ms


268796859830643408786989 * 271355115194767022865379

In [410]:
time(n.factor(algorithm="ecm"))

CPU times: user 107 ms, sys: 33.7 ms, total: 141 ms
Wall time: 8.19 s


268796859830643408786989 * 271355115194767022865379

3) Calculando novos primos p e q com metade do tamanho do anterior

* Tamanho dos novos primos em bits = 78 / 2 = 39

In [423]:
p2 = get_random_prime(2^(39 -1), 2^39 -1)
q2 = get_random_prime(2^(39 -1), 2^39 -1)
n2 = p2*q2
print(n2)
print(q2)
print(p2)

160742067700142190880643
501247683371
320683911433


In [424]:
time(n2.factor())

CPU times: user 11.8 ms, sys: 10.5 ms, total: 22.2 ms
Wall time: 23.4 ms


320683911433 * 501247683371

In [425]:
time(n2.factor(algorithm="qsieve"))

ValueError: n must have at least 40 digits

In [426]:
time(n2.factor(algorithm="ecm"))

CPU times: user 4.17 ms, sys: 16.2 ms, total: 20.4 ms
Wall time: 35.8 ms


320683911433 * 501247683371

4) Reduzindo o tamanho em bits de p e q em 1/4 do original, ou seja, 78 / 4 = 19.5, neste caso, o arredondamento sera considerado para cima, portanto, os primos agora terao 20 bits cada

In [415]:
p3 = get_random_prime(2^(20 -1), 2^20 -1)
q3 = get_random_prime(2^(20 -1), 2^20 -1)
n3 = p3*q3
print(n3)
print(q3)
print(p3)

479263354393
603607
793999


In [419]:
time(n3.factor())

CPU times: user 133 µs, sys: 0 ns, total: 133 µs
Wall time: 137 µs


603607 * 793999

In [420]:
time(n3.factor(algorithm="qsieve"))

CPU times: user 93 µs, sys: 11 µs, total: 104 µs
Wall time: 108 µs


603607 * 793999

In [421]:
time(n3.factor(algorithm="ecm"))

CPU times: user 149 µs, sys: 0 ns, total: 149 µs
Wall time: 148 µs


603607 * 793999

In [422]:
is_prime(65537)

True

#### Item 5 do exercicio 8
Para n com 156 bits temos
- PARI:   607 ms
- QSieve: 761 ms
- ECM:    8.19 s

Para n com 78 bits temos
- PARI:   23.4 ms
- QSieve: Error
- ECM:    35.8 ms

Para n com 40 bits (vide arredondamento para tamamho dos primos) temos
- PARI:   137 µs
- QSieve: 108 µs
- ECM:    148 µs

Deste modo, podemos determinar que o algoritmo padrao do PARI eh o mais eficiente para numeros com 156 bits e para 78 bits

Dado a caracteristica do algoritmo QSieve de ser mais adequado para numeros com mais de 100 digitos, pode-ser deduzir que a quantidade de 156 bits nao eh suficiente ainda para que se observe o ganho de desempenho do QSieve, alem disso, provavelmente devido a sua implementacao, o metodo gerou Erro ao tentar fatorar o numero n com 78 bits mas nao errou no calculo para 40 bits, e ainda por cima se demonstrou o mais performatico. Ou seja, podemos inferir que o algoritmo QSieve precisa de numeros com muito mais que 156 bits para destacar sua performance com relacao aos demais, mas o desempenho para o n de 40 bits revela que a implementacao provavelmente conta com algum caso especifico para numeros pequenos tambem.

O algotimo de curva eliptca, ECM, demonstrou ser o mais lento para todas as ocasioes, tendo seu resultado mais proximo ao dos demais para o n com 40 bits.

Esta comparacao permite determinar que, para casos gerais, o algoritmo padrao do PARI eh extremamente adequado, ja que apresenta uma variabilidade baixa em sua performance, portanto sua complexidade nao aumenta de maneira direta ou exponencial, o que pode contribuir para um ataque de fatoracao com mais eficiencia.
